In [ ]:
!pip install aiogram python-dotenv

In [ ]:
import os
from aiogram import Bot, Dispatcher, types
from aiogram.fsm.context import FSMContext
from aiogram.fsm.state import State, StatesGroup
from aiogram.fsm.storage.memory import MemoryStorage
from aiogram.filters import CommandStart, Command
from aiogram.types import BotCommand, ReplyKeyboardRemove
from dotenv import find_dotenv, load_dotenv

import nest_asyncio
import asyncio

# Применяем nest_asyncio
nest_asyncio.apply()

# Загрузка переменных окружения
load_dotenv(find_dotenv())

# Инициализация бота и диспетчера
bot = Bot(token="BOT_TOKEN")
storage = MemoryStorage()
dp = Dispatcher(storage=storage)

# Состояния для FSM (Finite State Machine)
class Form(StatesGroup):
    waiting_for_requirements = State()

# Команды бота
private_commands = [
    BotCommand(command='start', description='Старт'),
    BotCommand(command='help', description='Помощь'),
    BotCommand(command='find', description='Найти кандидатов'),
]

# Обработчик команды /start
@dp.message(CommandStart())
async def start_cmd(message: types.Message):
    await message.answer(
        "Привет! Я бот для поиска кандидатов. "
        "Отправь мне требования вакансии, и я найду подходящих специалистов."
    )

# Обработчик команды /help
@dp.message(Command('help'))
async def help_cmd(message: types.Message):
    commands_list = "\n".join([f"/{cmd.command} - {cmd.description}" for cmd in private_commands])
    await message.answer(f"Доступные команды:\n{commands_list}")

# Обработчик команды /find
@dp.message(Command('find'))
async def find_cmd(message: types.Message, state: FSMContext):
    await state.set_state(Form.waiting_for_requirements)
    await message.answer(
        "Пожалуйста, введите требования вакансии. "
        "Например: 'Опыт работы с Python, знание SQL.'"
    )

# Обработчик текстовых сообщений (требований вакансии)
@dp.message(Form.waiting_for_requirements)
async def process_requirements(message: types.Message, state: FSMContext):
    requirements = message.text

    # Ищем подходящих кандидатов
    job_emb = model.encode(requirements)
    faiss.normalize_L2(job_emb.reshape(1, -1))

    scores, indices = index.search(job_emb.reshape(1, -1).astype('float32'), 5)

    candidates = []
    for idx, score in zip(indices[0], scores[0]):
        if idx == -1:
            continue

        spec = specialists_data[idx]
        match_info = calculate_match({'requirements': requirements}, spec, skill_db)

        combined_score = 0.7 * score + 0.3 * (match_info['match_percent'] / 100)

        candidates.append({
            'name': spec['name'],
            'faiss_score': float(score),
            'combined_score': combined_score,
            **match_info
        })

    # Формируем ответ
    if not candidates:
        await message.answer("Подходящих кандидатов не найдено.")
        return

    response = "Найденные кандидаты:\n\n"
    for candidate in sorted(candidates, key=lambda x: -x['combined_score'])[:5]:
        response += (
            f"👤 **{candidate['name']}**\n"
            f"📊 Совпадение: {candidate['match_percent']:.2f}%\n"
            f"✅ Совпадающие навыки: {', '.join(candidate['matched_skills']) or 'нет'}\n"
            f"❌ Отсутствующие навыки: {', '.join(candidate['missing_skills']) or 'нет'}\n\n"
        )

    await message.answer(response, parse_mode="Markdown")
    await state.clear()

# Запуск бота
async def main():
    await bot.delete_webhook(drop_pending_updates=True)
    await bot.set_my_commands(commands=private_commands, scope=types.BotCommandScopeAllPrivateChats())
    await dp.start_polling(bot)

if __name__ == '__main__':
    asyncio.run(main())